<a href="https://colab.research.google.com/github/vikrammitra/prometheus/blob/main/AlgoTradingTest01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sourcing Alpaca Credentials

In [ ]:
credentials = open("/content/drive/MyDrive/Colab Notebooks/alpaca.txt","r")
lines = credentials.readlines()
consumer_key = lines[0].rstrip().replace("alpaca_key=","")
consumer_secret = lines[1].rstrip().replace("alpaca_secret=","")
#print(consumer_secret)


In [ ]:
!pip install alpaca-trade-api requests beautifulsoup4

In [ ]:
import pandas as pd
import alpaca_trade_api as tradeapi
from alpaca_trade_api import REST, Stream
import requests
from bs4 import BeautifulSoup
from datetime import datetime

API_KEY = consumer_key
API_SECRET = consumer_secret

def fetch_news_summary(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        summary = soup.find('p').text
        return summary.strip()
    except Exception as e:
        print(f"Error fetching news summary: {e}")
        return None

def get_news_data(ALPACA_API_KEY,ALPACA_SECRET_KEY,ticker):
    # Initialize Alpaca API
    api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)
    # Get the news data
    rest_client = REST(API_KEY, API_SECRET)
    news_list = rest_client.get_news(ticker)

    # Convert the news data into a pandas DataFrame
    data = {
        'Title': [],
        'Published': [],
        'Source': [],
        'Url': [],
        'Summary': []
    }

    for news in news_list:
        data['Title'].append(news.headline)
        data['Published'].append(news.created_at.strftime('%Y-%m-%d %H:%M:%S'))
        data['Source'].append(news.source)
        data['Url'].append(news.url)
        data['Summary'].append(fetch_news_summary(news.url))

    df = pd.DataFrame(data)
    return df

if __name__ == "__main__":
    news_df = get_news_data(API_KEY,API_SECRET,"AAPL")

In [ ]:
news_df

,Title,Published,Source,Url,Summary
0,US Stocks Brace For Sluggish Start To Fresh We...,2023-07-31 11:01:36,benzinga,https://www.benzinga.com/news/earnings/23/07/3...,Early indications suggest a nervous start by W...
1,Tim Cook Gets Stinging Letter From Congressman...,2023-07-31 10:53:06,benzinga,https://www.benzinga.com/markets/cryptocurrenc...,"In a bipartisan effort, Representative Gus Bil..."
2,Can Apple Satisfy Wall Street? Top Analyst Say...,2023-07-31 10:05:47,benzinga,https://www.benzinga.com/analyst-ratings/analy...,"Apple, Inc. AAPL is scheduled to report its fi..."
3,"It's Apple, Not Your Kids: Apple Confirms Bug ...",2023-07-31 09:54:23,benzinga,https://www.benzinga.com/news/23/07/33465360/i...,A bug in Apple Inc.'s AAPL Screen Time parenta...
4,"Piper Sandler Maintains Overweight on Apple, R...",2023-07-31 09:53:23,benzinga,https://www.benzinga.com/news/23/07/33465354/p...,Never miss a trade again with the fastest news...
5,Monitor Your iPhone Battery Like A Pro: Add Ba...,2023-07-31 06:08:34,benzinga,https://www.benzinga.com/news/23/07/33464138/m...,Have you ever found yourself anxiously glancin...
6,Apple's iPhone 15 To Have Biggest Upgrade In 3...,2023-07-30 16:34:11,benzinga,https://www.benzinga.com/news/23/07/33460903/a...,The countdown to the launch of the next iterat...
7,"Wall Street Journal Reported Saturday, Apple A...",2023-07-30 13:35:29,benzinga,https://www.benzinga.com/news/23/07/33460267/w...,Never miss a trade again with the fastest news...
8,"MagSafe Discount, Netflix's Decision On Apple ...",2023-07-30 13:00:16,benzinga,https://www.benzinga.com/news/23/07/33460043/m...,"This week in the Appleverse, we saw a range of..."
9,"Benzinga Bulls And Bears: Tesla, Apple, AMC, A...",2023-07-29 15:38:14,benzinga,https://www.benzinga.com/trading-ideas/long-id...,Benzinga examined the prospects for many inves...
